In [1]:
import sympy as sym
from sympy import simplify
import numpy as np
import utils



In [2]:
t = sym.symbols("t", real=True)
u = sym.Function("u", real=True)(t)
v = sym.Function("v", real=True)(t)
w = sym.Function("w", real=True)(t)
p = sym.Function("p", real=True)(t)
q = sym.Function("q", real=True)(t)
r = sym.Function("r",real=True)(t)
phi = sym.Function(r"\phi", real=True)(t)
theta = sym.Function(r"\theta", real=True)(t)
psi = sym.Function(r"\psi", real=True)(t)

X = sym.Function("X")(t)
Y = sym.Function("Y")(t)
Z = sym.Function("Y")(t)
K = sym.Function("K")(t)
M = sym.Function("M")(t)
N = sym.Function("N")(t)

x = sym.Matrix([u, v, w, p, q, r, phi, theta, psi])
tau = sym.Matrix([X, Y, Z, K, M, N])


In [3]:
## Hydrodynamic parameters
W = sym.symbols("W")
B = sym.symbols("B")
m = sym.symbols("m")
Ix = sym.symbols("Ix")
Iy = sym.symbols("Iy")
Iz = sym.symbols("Iz")

Xdu = sym.symbols("X_{\dot{u}}")
Ydv = sym.symbols("Y_{\dot{v}}")
Zdw = sym.symbols("Z_{\dot{w}}")
Kdp = sym.symbols("K_{\dot{p}}")
Mdq = sym.symbols("M_{\dot{q}}")
Ndr = sym.symbols("N_{\dot{r}}")

Xu = sym.symbols("X_u")
Yv = sym.symbols("Y_v")
Zw = sym.symbols("Z_w")
Kp = sym.symbols("K_p")
Mq = sym.symbols("M_q")
Nr = sym.symbols("N_r")

Xabs_u = sym.symbols("X_{u|u|}")
Yabs_v = sym.symbols("Y_{v|v|}")
Zabs_w = sym.symbols("Z_{w|w|}")
Kabs_p = sym.symbols("K_{p|p|}")
Mabs_q = sym.symbols("M_{q|q|}")
Nabs_r = sym.symbols("N_{r|r|}")


In [4]:
xb, yb, zb = sym.symbols(r"x_b y_b z_b")
xg, yg, zg = sym.symbols(r"x_g y_g z_g")
rg = sym.Matrix([xg, yg, zg])
rb = sym.Matrix([xb, yb, zb])


In [5]:
def Rzyx(phi, theta, psi):
    """
    R = Rzyx(phi,theta,psi) computes the Euler angle rotation matrix R in SO(3)
    using the zyx convention
    """

    cphi = sym.cos(phi)
    sphi = sym.sin(phi)
    cth = sym.cos(theta)
    sth = sym.sin(theta)
    cpsi = sym.cos(psi)
    spsi = sym.sin(psi)

    R = sym.Matrix([
        [cpsi * cth, -spsi * cphi + cpsi * sth * sphi, spsi * sphi + cpsi * cphi * sth],
        [spsi * cth, cpsi * cphi + sphi * sth * spsi, -cpsi * sphi + sth * spsi * cphi],
        [-sth, cth * sphi, cth * cphi]])

    return R


In [6]:
def Tzyx(phi, theta):
    """
    T = Tzyx(phi,theta) computes the Euler angle attitude
    transformation matrix T using the zyx convention
    """

    cphi = sym.cos(phi)
    sphi = sym.sin(phi)
    cth = sym.cos(theta)
    sth = sym.sin(theta)

    try:
        T = sym.Matrix([
            [1, sphi * sth / cth, cphi * sth / cth],
            [0, cphi, -sphi],
            [0, sphi / cth, cphi / cth]])

    except ZeroDivisionError:
        print("Tzyx is singular for theta = +-90 degrees.")

    return T

Tzyx(phi, theta)


Matrix([
[1, sin(\phi(t))*sin(\theta(t))/cos(\theta(t)), sin(\theta(t))*cos(\phi(t))/cos(\theta(t))],
[0,                               cos(\phi(t)),                              -sin(\phi(t))],
[0,                sin(\phi(t))/cos(\theta(t)),                cos(\phi(t))/cos(\theta(t))]])

In [7]:
def J(eta):
    a = np.block([[Rzyx(eta[0], eta[1], eta[2]), np.zeros(((3, 3)))],
                  [np.zeros((3, 3)), Tzyx(eta[1], eta[2])]])
    return sym.Matrix(a)

J(x[6:])
    


Matrix([
[cos(\psi(t))*cos(\theta(t)), sin(\phi(t))*sin(\theta(t))*cos(\psi(t)) - sin(\psi(t))*cos(\phi(t)),  sin(\phi(t))*sin(\psi(t)) + sin(\theta(t))*cos(\phi(t))*cos(\psi(t)), 0,                                        0,                                        0],
[sin(\psi(t))*cos(\theta(t)), sin(\phi(t))*sin(\psi(t))*sin(\theta(t)) + cos(\phi(t))*cos(\psi(t)), -sin(\phi(t))*cos(\psi(t)) + sin(\psi(t))*sin(\theta(t))*cos(\phi(t)), 0,                                        0,                                        0],
[            -sin(\theta(t)),                                          sin(\phi(t))*cos(\theta(t)),                                           cos(\phi(t))*cos(\theta(t)), 0,                                        0,                                        0],
[                          0,                                                                    0,                                                                     0, 1, sin(\psi(t))*sin(\theta(t))/cos(\psi(t))

In [8]:
MRB = sym.Matrix([[m, 0, 0, 0, 0, 0],
                 [0, m, 0, 0, 0, 0],
                 [0, 0, m, 0, 0, 0],
                 [0, 0, 0, Ix, 0, 0],
                 [0, 0, 0, 0, Iy, 0],
                 [0, 0, 0, 0, 0, Iz]])
MA = -sym.diag(Xdu, Ydv, Zdw, Kdp, Mdq, Ndr)

M_ = MA + MRB


In [9]:
def gvect(W, B, theta, phi, r_bg, r_bb):
    """
    g = gvect(W,B,theta,phi,r_bg,r_bb) computes the 6x1 vector of restoring
    forces about an arbitrarily point CO for a submerged body.

    Inputs:
        W, B: weight and buoyancy (kg)
        phi,theta: roll and pitch angles (rad)
        r_bg = [x_g y_g z_g]: location of the CG with respect to the CO (m)
        r_bb = [x_b y_b z_b]: location of the CB with respect to th CO (m)

    Returns:
        g: 6x1 vector of restoring forces about CO
    """

    sth = sym.sin(theta)
    cth = sym.cos(theta)
    sphi = sym.sin(phi)
    cphi = sym.cos(phi)
    xb, yb, zb = r_bb

    g = sym.Matrix([
        (W - B) * sth,
        -(W - B) * cth * sphi,
        -(W - B) * cth * cphi,
        yb * B * cth * cphi - zb * B * cth * sphi,
        -zb * B * sth - xb * cth * cphi,
        xb * B * cth * sth + yb * B * sth,
    ])

    return g
g = gvect(W, B, x[6], x[7], rg, rb)


In [10]:
D = -sym.diag(Xu, Yv, Zw, Kp, Mq, Nr)

D = D - sym.diag(Xabs_u * abs(x[0]),
                 Yabs_v * abs(x[1]),
                 Zabs_w * abs(x[2]),
                 Kabs_p * abs(x[3]),
                 Mabs_q * abs(x[4]),
                 Nabs_r * abs(x[5]))



In [11]:
def Smtrx(a):
    """
    S = Smtrx(a) computes the 3x3 vector skew-symmetric matrix S(a) = -S(a)'.
    The cross product satisfies: a x b = S(a)b.
    """

    S = sym.Matrix([
        [0, -a[2], a[1]],
        [a[2], 0, -a[0]],
        [-a[1], a[0], 0]])

    return S


def m2c(M, nu):
    """
    C = m2c(M,nu) computes the Coriolis and centripetal matrix C from the
    mass matrix M and generalized velocity vector nu (Fossen 2021, Ch. 3)
    """

    M = sym.Rational(1,2) * (M + M.transpose())  # systematization of the inertia matrix

    M11 = M[0:3, 0:3]
    M12 = M[0:3, 3:6]
    M21 = M12.T
    M22 = M[3:6, 3:6]

    nu1 = sym.Matrix(nu[0:3])
    nu2 = sym.Matrix(nu[3:6])
    #dt_dnu1 = np.matmul(M11, nu1) + np.matmul(M12, nu2)
    #dt_dnu2 = np.matmul(M21, nu1) + np.matmul(M22, nu2)
    dt_dnu1 = M11 @ nu1 + M12 @ nu2
    dt_dnu2 = M21 @ nu1 + M22 @ nu2
    # C  = [  zeros(3,3)      -Smtrx(dt_dnu1)
    #      -Smtrx(dt_dnu1)  -Smtrx(dt_dnu2) ]
    C = sym.zeros(6, 6)
    C[0:3, 3:6] = -Smtrx(dt_dnu1)
    C[3:6, 0:3] = -Smtrx(dt_dnu1)
    C[3:6, 3:6] = -Smtrx(dt_dnu2)

    return C


C = m2c(M_, sym.Matrix(x[:6]))

In [12]:
v = sym.Matrix(x[:6])

vM = tau - D @ v - C @ v - g
Theta = Tzyx(x[6], x[7]) @ sym.Matrix(x[3:6])
M_
M_9x9 = M_.row_join(sym.zeros(6,3)).col_join(sym.zeros(3,6).row_join(sym.eye(3)))
t
f = M_9x9.inv() @ sym.Matrix([vM, Theta])
print(type(f))
f = sym.simplify(f)

<class 'sympy.matrices.dense.MutableDenseMatrix'>


In [133]:
f

Matrix([
[                                                                                                                ((-B + W)*sin(\phi(t)) - (X_u + X_{u|u|}*Abs(u(t)))*u(t) + (Y_{\dot{v}} - m)*r(t)*v(t) - (Z_{\dot{w}} - m)*q(t)*w(t) - X(t))/(X_{\dot{u}} - m)],
[                                                                                                  ((B - W)*sin(\theta(t))*cos(\phi(t)) - (X_{\dot{u}} - m)*r(t)*u(t) - (Y_v + Y_{v|v|}*Abs(v(t)))*v(t) + (Z_{\dot{w}} - m)*p(t)*w(t) - Y(t))/(Y_{\dot{v}} - m)],
[                                                                                                  ((B - W)*cos(\phi(t))*cos(\theta(t)) + (X_{\dot{u}} - m)*q(t)*u(t) - (Y_{\dot{v}} - m)*p(t)*v(t) - (Z_w + Z_{w|w|}*Abs(w(t)))*w(t) - Y(t))/(Z_{\dot{w}} - m)],
[(-B*y_b*cos(\phi(t))*cos(\theta(t)) + B*z_b*sin(\theta(t))*cos(\phi(t)) + (Iy - M_{\dot{q}})*q(t)*r(t) - (Iz - N_{\dot{r}})*q(t)*r(t) + (K_p + K_{p|p|}*Abs(p(t)))*p(t) - (Y_{\dot{v}} - m)*v(t)*w(t) + (Z_{\dot{w}} - m

In [135]:
simplify(Theta)

Matrix([
[p(t) + q(t)*sin(\phi(t))*tan(\theta(t)) + r(t)*cos(\phi(t))*tan(\theta(t))],
[                                     q(t)*cos(\phi(t)) - r(t)*sin(\phi(t))],
[                    (q(t)*sin(\phi(t)) + r(t)*cos(\phi(t)))/cos(\theta(t))]])

In [38]:
AM = simplify(sym.Matrix([vM, Theta]).jacobian(x))
AM

Matrix([
[Piecewise((X_u, Eq(u(t), 0)), (X_u + 2*X_{u|u|}*u(t)**2/Abs(u(t)), True)),                                                   (-Y_{\dot{v}} + m)*r(t),                                                    (Z_{\dot{w}} - m)*q(t),                                                                         0,                                                    (Z_{\dot{w}} - m)*w(t),                                                   (-Y_{\dot{v}} + m)*v(t),                                     (B - W)*cos(\phi(t)),                                                                        0, 0],
[                                                   (X_{\dot{u}} - m)*r(t), Piecewise((Y_v, Eq(v(t), 0)), (Y_v + 2*Y_{v|v|}*v(t)**2/Abs(v(t)), True)),                                                   (-Z_{\dot{w}} + m)*p(t),                                                   (-Z_{\dot{w}} + m)*w(t),                                                                         0,                             

In [132]:
AM[8, 8]

0